In [23]:
import os 
import pandas as pd
import numpy as np

In [24]:
crt_dir = os.path.abspath("")
data_dir = os.path.abspath("data")
idx000300_dir = os.path.join(data_dir, "000300Weight_of_Constituent_Stock")

# Data Preparation

todo: description of our index data. why 000300? what did we do?

The historical daily constituent data of index 000300 from 2009-12 to current time we downloaded from CSMAR are seperated into 3 files due to the CSMAR's 5-year data maximum query policy. The following cell combine all data files and read them into RAM:

In [25]:
def genIDX_all(dir, namelist, output_name=None, force=False):
    if not os.path.lexists(os.path.join(dir, output_name)) or force:
        dflist = []
        for name in namelist:
            file = os.path.join(dir, name)
            dflist.append(pd.read_csv(file, header=0, index_col=1, parse_dates=True))
        df = pd.concat(dflist)
        df.to_csv(os.path.join(dir, output_name))
        return df
    else:
        return pd.read_csv(os.path.join(dir, output_name), header=0, index_col=0, parse_dates=True)

In [26]:
idx_filelist = list(filter(lambda file: file.startswith("IDX_Smprat_"), os.listdir(idx000300_dir)))
idx_filelist

['IDX_Smprat_1.csv', 'IDX_Smprat_2.csv', 'IDX_Smprat_3.csv']

In [27]:
df = genIDX_all(idx000300_dir, idx_filelist, 'All_IDX_Smprat.csv')
df

,Indexcd,Stkcd,Constdnme_en,Weight
Enddt,,,,
2009-12-01,300,600547,SDHJ,0.540
2009-12-01,300,600548,SGS,0.030
2009-12-01,300,600549,XMWY,0.080
2009-12-01,300,600550,TWBB,0.360
2009-12-01,300,600569,AYGT,0.100
...,...,...,...,...
2022-12-08,300,601186,ZGTJ,0.220
2022-12-08,300,2555,SQHY,0.125
2022-12-08,300,600919,JSYH,0.478


**Enable the following cell if you need to regenerate the combined dataset:**

In [28]:
# df = genIDX_all(idx000300_dir, idx_filelist, 'All_IDX_Smprat.csv', force=True)

**Select the stocks that are in 000300 portfolio during 2019-12-01 to 2022-12-08**

In [35]:
numOfDays = df.index.unique().size
stock_mask = df.groupby("Stkcd")["Indexcd"].count() >0#==numOfDays
stock_list = stock_mask.index[stock_mask]
stock_list = stock_list[(680000>stock_list)&(stock_list>=600000)].values
print(stock_mask.index)

Int64Index([     1,      2,      8,      9,     12,     21,     24,     27,
                31,     39,
            ...
            688036, 688065, 688111, 688126, 688169, 688363, 688396, 688561,
            688599, 688981],
           dtype='int64', name='Stkcd', length=757)


**Now we have obtained the list of stocks of interest. Next step is to obtain the monthly return rates of these stocks:**

In [30]:
TRD_df = pd.read_csv(os.path.join(data_dir, "TRD_Mnth.csv"),
                   header=0, index_col=1, parse_dates=True)
stock_dict = stock_mask.to_dict()
TRD_df = TRD_df[TRD_df['Stkcd'].apply(lambda x: stock_dict.get(x, False))]


**Store the monthly return rates of our selected stockes:**

In [31]:
R_df_list = []
for stock in stock_list:
    stock_df = TRD_df[TRD_df['Stkcd'] == stock]
    tmp = pd.DataFrame(stock_df['Mretwd'], index=stock_df.index)
    tmp.columns = [stock]
    R_df_list.append(tmp)


In [32]:
R_df = pd.concat(R_df_list, axis=1)
#R_df = R_df.fillna(-999.0)
R_df.to_csv(os.path.join(data_dir, "Monthly_Return_Rates.csv"))
R_df

,600000,600001,600004,600005,600006,600008,600009,600010,600011,600015,...,603806,603833,603858,603882,603885,603899,603939,603986,603993,605499
Trdmnt,,,,,,,,,,,,,,,,,,,,,
2009-12-01,-0.014986,-0.034672,0.026289,0.019704,0.037821,-0.016304,0.139307,-0.027254,-0.007435,0.098143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-01,-0.095436,NaN,0.055172,-0.182367,-0.151603,-0.085635,0.064868,-0.131466,-0.081149,-0.132850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-01,0.057085,NaN,0.099907,0.005908,0.049828,0.225076,0.031806,0.029777,0.002717,0.018570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-01,0.098361,NaN,-0.033956,-0.017621,0.029460,-0.043157,-0.031870,0.019277,-0.008130,0.168642,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-01,-0.096137,NaN,-0.097540,-0.159940,-0.033386,-0.073454,-0.134377,-0.073286,-0.083333,-0.007800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-01,-0.042936,NaN,-0.131455,NaN,-0.044108,0.013793,-0.090300,-0.038298,-0.002841,-0.017275,...,0.064255,-0.189507,-0.039574,-0.143560,-0.172874,-0.190264,-0.020822,-0.168343,-0.046848,-0.104179
2022-08-01,0.001377,NaN,0.043243,NaN,-0.138268,-0.044218,0.093835,-0.110619,0.165242,0.003906,...,-0.069554,0.083707,-0.030393,-0.042510,0.055107,-0.022682,0.032283,-0.019447,-0.087199,-0.005031
2022-09-01,-0.031637,NaN,0.055514,NaN,-0.072934,-0.074733,0.024105,-0.084577,-0.072127,-0.021401,...,-0.180025,-0.129425,-0.026111,-0.049348,-0.035669,0.015998,-0.068165,-0.191601,-0.040650,-0.068356


In [33]:
rf = pd.read_csv(os.path.join(data_dir, "Monthly_rf_Rates.csv"), index_col=0, parse_dates=True)
R_excess_df = R_df - rf.values
R_excess_df.to_csv(os.path.join(data_dir, "Monthly_Excess_Return_Rates.csv"))
R_excess_df

,600000,600001,600004,600005,600006,600008,600009,600010,600011,600015,...,603806,603833,603858,603882,603885,603899,603939,603986,603993,605499
Trdmnt,,,,,,,,,,,,,,,,,,,,,
2009-12-01,-0.016842,-0.036528,0.024433,0.017848,0.035965,-0.018160,0.137451,-0.029110,-0.009291,0.096287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-01,-0.097292,NaN,0.053316,-0.184223,-0.153459,-0.087491,0.063012,-0.133322,-0.083005,-0.134706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-01,0.055229,NaN,0.098051,0.004052,0.047972,0.223220,0.029950,0.027921,0.000861,0.016714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-01,0.096505,NaN,-0.035812,-0.019477,0.027604,-0.045013,-0.033726,0.017421,-0.009986,0.166786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-01,-0.097993,NaN,-0.099396,-0.161796,-0.035242,-0.075310,-0.136233,-0.075142,-0.085189,-0.009656,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-01,-0.044177,NaN,-0.132696,NaN,-0.045349,0.012552,-0.091541,-0.039539,-0.004082,-0.018516,...,0.063014,-0.190748,-0.040815,-0.144801,-0.174115,-0.191505,-0.022063,-0.169584,-0.048089,-0.105420
2022-08-01,0.000136,NaN,0.042002,NaN,-0.139509,-0.045459,0.092594,-0.111860,0.164001,0.002665,...,-0.070795,0.082466,-0.031634,-0.043751,0.053866,-0.023923,0.031042,-0.020688,-0.088440,-0.006272
2022-09-01,-0.032878,NaN,0.054273,NaN,-0.074175,-0.075974,0.022864,-0.085818,-0.073368,-0.022642,...,-0.181266,-0.130666,-0.027352,-0.050589,-0.036910,0.014757,-0.069406,-0.192842,-0.041891,-0.069597
